In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Read the table from wikipedia

In [6]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [7]:
#select only the desired table
df=df[0]

### Remove Boroughs marked as "Not assigned"

In [22]:
df = df.loc[df['Borough']!='Not assigned']

In [24]:
df['Neighborhood'] = np.where(df['Neighborhood']=='Not assigned', df['Borough'], df['Neighborhood'])